## Interfaz de Probabilidad Condicional


---

### Equipo:
- Gutiérrez Pérez Gabriela G.
- Loeza Sebastián.
- Ramos García Luis Gerardo.
- Rico Mendoza Josué.

-----

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import re
import pandas as pd
from nltk.util import ngrams
import os

# Lista para almacenar las rutas de archivos
file_paths = []

# Función para agregar archivo ingresado manualmente
def add_file(entry, file_listbox):
    file_path = entry.get()
    if file_path:
        if file_path not in file_paths:
            file_paths.append(file_path)
            file_listbox.insert(tk.END, file_path)
            entry.delete(0, tk.END)  # Limpiar cuadro de texto
        else:
            messagebox.showinfo("Info", "El archivo ya está en la lista.")
    else:
        messagebox.showwarning("Advertencia", "Por favor, ingresa una ruta válida o busque el archivo.")

# Función para abrir el gestor de archivos y colocar la ruta en el cuadro de texto
def browse_files(entry):
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        entry.delete(0, tk.END)  # Limpiar cuadro de texto antes de poner la nueva ruta
        entry.insert(0, file_path)  # Colocar la ruta seleccionada en el cuadro de texto

# Función para eliminar archivo seleccionado de la lista
def delete_file(file_listbox):
    selected_file_index = file_listbox.curselection()
    if selected_file_index:
        selected_file = file_listbox.get(selected_file_index)
        file_paths.remove(selected_file)
        file_listbox.delete(selected_file_index)
    else:
        messagebox.showwarning("Advertencia", "Por favor, selecciona un archivo para eliminar.")

# Función para tokenizar el texto
def tokenizeCorpus(corpus):
    tokenizer = re.compile(r'\w+|[^\w\s]')  # Separar palabras y signos
    return tokenizer.findall(corpus.lower())

# Función para generar N-gramas
def generateNgrams(tokenizedText, n):
    return list(ngrams(tokenizedText, n))

# Función para generar bigramas o trigramas
def generate_ngrams(entry_text, ngram_type):
    # Añadir etiquetas de apertura ($) y cierre (&)
    corpus = f"${entry_text}&"
    
    tokenized_text = tokenizeCorpus(corpus)

    if ngram_type == 'bigramas':
        ngram_data = generateNgrams(tokenized_text, 2)
        df = pd.DataFrame(ngram_data, columns=['Term1', 'Term2'])
    else:
        ngram_data = generateNgrams(tokenized_text, 3)
        df = pd.DataFrame(ngram_data, columns=['Term1', 'Term2', 'Term3'])
    
    # Guardar el archivo CSV
    df.to_csv(f'{ngram_type}_ejemplo.csv', index=False)
    messagebox.showinfo("Guardado", f"El archivo {ngram_type}_ejemplo.csv ha sido guardado.")

def calculate_joint_probability(ngram_type):

    # Elegir el archivo CSV según el tipo de n-grama
    if ngram_type == 'bigramas':
        ngram_file = 'bigramas_ejemplo.csv'
    else:
        ngram_file = 'trigramas_ejemplo.csv'
    
    # Leer el archivo de n-gramas
    ngram_df = pd.read_csv(ngram_file)
    results = {}  # Diccionario para almacenar los resultados

    # Iterar sobre cada archivo proporcionado
    for file_path in file_paths:
        total_probability = 1.0  # Inicializar probabilidad total para este archivo
        match_count = 0  # Contador de coincidencias de bigramas/trigramas
        
        # Leer el archivo actual
        try:
            data = pd.read_csv(file_path)
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo leer el archivo {file_path}: {str(e)}")
            continue  # Pasar al siguiente archivo si hay un error

        # Iterar sobre cada n-grama en el DataFrame de n-gramas
        for index, row in ngram_df.iterrows():
            if ngram_type == 'bigramas':
                term1 = row['Term1']
                term2 = row['Term2']
                
                # Buscar la probabilidad del bigrama en el archivo actual
                match = data[(data['Term1'] == term1) & (data['Term2'] == term2)]
                if not match.empty:
                    bigram_prob = match['BigramProb'].values[0]
                    total_probability *= bigram_prob  # Multiplicar la probabilidad total
                    match_count += 1  # Incrementar el contador de coincidencias

            else:  # Para trigramas
                term1 = row['Term1']
                term2 = row['Term2']
                term3 = row['Term3']
                
                # Buscar la probabilidad del trigram en el archivo actual
                match = data[(data['Term1'] == term1) & (data['Term2'] == term2) & (data['Term3'] == term3)]
                if not match.empty:
                    trigram_prob = match['TrigramProb'].values[0]
                    total_probability *= trigram_prob  # Multiplicar la probabilidad total
                    match_count += 1  # Incrementar el contador de coincidencias

        # Si no se encontró ningún bigrama/trigrama, asignar probabilidad conjunta de 0
        if match_count == 0:
            total_probability = 0

        # Almacenar el resultado para este archivo
        results[os.path.basename(file_path)] = total_probability

  # Mostrar resultados en el cuadro de texto
    #result_message = "\n".join([f"{file}: {probability:.4f}" for file, probability in results.items()])
    #result_text.delete(1.0, tk.END)  # Limpiar el cuadro de texto antes de mostrar nuevos resultados
    #result_text.insert(tk.END, f"Las probabilidades conjuntas son:\n{result_message}")

    # Ordenar los resultados de mayor a menor por la probabilidad
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

    # Mostrar los resultados ordenados en el cuadro de texto
    result_message = "\n".join([f"{file}: {probability:.8f}" for file, probability in sorted_results])
    result_text.delete(1.0, tk.END)  # Limpiar el cuadro de texto antes de mostrar nuevos resultados
    result_text.insert(tk.END, f"Las probabilidades conjuntas son:\n{result_message}")


# Función para manejar el clic en el botón de probabilidad conjunta
def handle_joint_probability():
    ngram_type = ngram_choice.get()
    entry_text = text_entry.get()

    # Generar n-gramas primero
    generate_ngrams(entry_text, ngram_type)

    # Luego calcular la probabilidad conjunta
    calculate_joint_probability(ngram_type)

# Crear la ventana principal
def create_window():
    global text_entry, ngram_choice, result_text

    root = tk.Tk()
    root.title("Cargar CSV y Generar N-gramas")

    # Cuadro de texto para ingresar la ruta
    entry = tk.Entry(root, width=50)
    entry.grid(row=0, column=0, padx=10, pady=10)

    # Botón "Agregar" para agregar el archivo
    add_button = tk.Button(root, text="Agregar", command=lambda: add_file(entry, file_listbox))
    add_button.grid(row=0, column=1, padx=10, pady=10)

    # Botón "Buscar" para abrir el gestor de archivos y colocar la ruta en el cuadro de texto
    search_button = tk.Button(root, text="Buscar", command=lambda: browse_files(entry))
    search_button.grid(row=0, column=2, padx=10, pady=10)

    # Listbox para mostrar las rutas de los archivos cargados
    file_listbox = tk.Listbox(root, width=70, height=10)
    file_listbox.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

    # Botón "Eliminar" para eliminar el archivo seleccionado
    delete_button = tk.Button(root, text="Eliminar", command=lambda: delete_file(file_listbox))
    delete_button.grid(row=2, column=0, columnspan=3, padx=10, pady=10)

    # Cuadro de texto para ingresar texto
    text_entry = tk.Entry(root, width=50)
    text_entry.grid(row=3, column=0, padx=10, pady=10, columnspan=3)

    # Radio buttons para seleccionar entre bigramas y trigramas
    ngram_choice = tk.StringVar(value='bigramas')
    bigram_radio = tk.Radiobutton(root, text="Bigramas", variable=ngram_choice, value='bigramas')
    bigram_radio.grid(row=4, column=0, padx=10, pady=10)
    
    trigram_radio = tk.Radiobutton(root, text="Trigramas", variable=ngram_choice, value='trigramas')
    trigram_radio.grid(row=4, column=1, padx=10, pady=10)

    # Botón "Probabilidad Conjunta"
    joint_prob_button = tk.Button(root, text="Probabilidad Conjunta", command=handle_joint_probability)
    joint_prob_button.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

    # Cuadro de texto para mostrar resultados
    result_text = tk.Text(root, width=70, height=10)
    result_text.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

    root.mainloop()

# Ejecutar la aplicación
create_window()


2024-11-05 20:26:43.585 python[21440:1914984] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-05 20:26:43.585 python[21440:1914984] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2024-11-05 20:26:48.717 python[21440:1914984] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
